In [1]:
from mylib.models import densesharp, metrics, losses,densenet
from mylib.utils.misc import rotation, reflection, crop, random_center, _triple

from keras.callbacks import ModelCheckpoint, CSVLogger, TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.optimizers import Adam,Adamax,Nadam
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv3D, MaxPool3D
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

class MixupGenerator():
    def __init__(self, X_train, y_train, batch_size=32, alpha=0.2, shuffle=True, datagen=None):
        X_train_new = []
        for i in range(X_train.shape[0]):
                try:
                    X_train_new = np.append(X_train_new, np.expand_dims(Transform(32,5)(X_train[i]), axis=0), axis=0)                  
                except ValueError:
                    X_train_new = np.expand_dims(Transform(32,5)(X_train[0]), axis=0)
        self.X_train = X_train_new
        self.y_train = y_train
        self.batch_size = batch_size
        self.alpha = alpha
        self.shuffle = shuffle
        self.sample_num = len(X_train)
        self.datagen = datagen

    def __call__(self):
        while True:
            indexes = self.__get_exploration_order()
            itr_num = int(len(indexes) // (self.batch_size * 2))

            for i in range(itr_num):
                batch_ids = indexes[i * self.batch_size * 2:(i + 1) * self.batch_size * 2]
                X, y = self.__data_generation(batch_ids)

                yield X, y

    def __get_exploration_order(self):
        indexes = np.arange(self.sample_num)

        if self.shuffle:
            np.random.shuffle(indexes)

        return indexes

    def __data_generation(self, batch_ids):
        _, h, w, c = self.X_train.shape
        l = np.random.beta(self.alpha, self.alpha, self.batch_size)
        X_l = l.reshape(self.batch_size, 1, 1, 1)
        y_l = l.reshape(self.batch_size, 1)

        X1 = self.X_train[batch_ids[:self.batch_size]]
        X2 = self.X_train[batch_ids[self.batch_size:]]
        X = X1 * X_l + X2 * (1 - X_l)
        X = np.expand_dims(X,axis=-1)

        if self.datagen:
            for i in range(self.batch_size):
                X[i] = self.datagen.random_transform(X[i])
                X[i] = self.datagen.standardize(X[i])

        if isinstance(self.y_train, list):
            y = []

            for y_train_ in self.y_train:
                y1 = y_train_[batch_ids[:self.batch_size]]
                y2 = y_train_[batch_ids[self.batch_size:]]
                y.append(y1 * y_l + y2 * (1 - y_l))
        else:
            y1 = self.y_train[batch_ids[:self.batch_size]]
            y2 = self.y_train[batch_ids[self.batch_size:]]
            y = y1 * y_l + y2 * (1 - y_l)

        return X, y

class Transform:
    '''The online data augmentation, including:
    1) random move the center by `move`
    2) rotation 90 degrees increments
    3) reflection in any axis
    '''

    def __init__(self, size, move):
        self.size = _triple(size)
        self.move = move

    def __call__(self, arr, aux=None):
        shape = arr.shape
        if self.move is not None:
            center = random_center(shape, self.move)
            arr_ret = crop(arr, center, self.size)
            angle = np.random.randint(4, size=3)
            arr_ret = rotation(arr_ret, angle=angle)
            axis = np.random.randint(4) - 1
            arr_ret = reflection(arr_ret, axis=axis)
            #arr_ret = np.expand_dims(arr_ret, axis=0)
            if aux is not None:
                aux_ret = crop(aux, center, self.size)
                aux_ret = rotation(aux_ret, angle=angle)
                aux_ret = reflection(aux_ret, axis=axis)
                aux_ret = np.expand_dims(aux_ret, axis=0)
                return arr_ret, aux_ret
            return arr_ret
        else:
            center = np.array(shape) // 2
            arr_ret = crop(arr, center, self.size)
            arr_ret = np.expand_dims(arr_ret, axis=0)
            if aux is not None:
                aux_ret = crop(aux, center, self.size)
                #aux_ret = np.expand_dims(aux_ret, axis=0)
                return arr_ret, aux_ret
            return arr_ret


Using TensorFlow backend.


In [2]:



voxel_train = []
seg_train = []
for i in tqdm(range(584), desc='reading'):
    try:
        data = np.load('data/train_val/candidate{}.npz'.format(i))
    except FileNotFoundError:
        continue
    try:
        voxel_train = np.append(voxel_train, np.expand_dims(data['voxel'], axis=0), axis=0)
        seg_train = np.append(seg_train, np.expand_dims(data['seg'], axis=0), axis=0)
    except ValueError:
        voxel_train = np.expand_dims(data['voxel'], axis=0)
        seg_train = np.expand_dims(data['seg'], axis=0)



#voxel_train=voxel_train.astype('float32')
training_batch_size = voxel_train.shape[0]#465
train_label = pd.read_csv('data/train_val.csv').values[:, 1].astype(int)
train_label = to_categorical(train_label, 2)
voxel_train=voxel_train*seg_train
x_train,x_val,y_train, y_val =train_test_split(voxel_train,train_label,test_size=100,shuffle=True,stratify=train_label)
tmp_val=[]
tmp_val = np.expand_dims(crop(x_val[0],(50,50,50),(32,32,32)),axis=0)
for i in range(x_val.shape[0]-1):
    tmp_val = np.append(tmp_val,np.expand_dims(crop(x_val[i+1],(50,50,50),(32,32,32)),axis=0),axis=0)
x_val = tmp_val
x_val = np.expand_dims(x_val,axis=-1)
print(x_val.shape)

reading: 100%|██████████| 584/584 [00:19<00:00, 30.53it/s]


(100, 32, 32, 32, 1)


In [3]:
#读取测试数据
voxel_test = []		#设置测试数据的voxel_test
seg_test = []		#设置测试数据的seg_test

for i in tqdm(range(584), desc='reading test_data'):	#展示写入测试数据的进度
    try:
        tmp = np.load('data/test/candidate{}.npz'.format(i))	#依次读取测试数据中的candidate{i}文件
    except FileNotFoundError:									#无该文件时直接进入下一次循环
        continue
    try:
        voxel_test = np.append(voxel_test, np.expand_dims(tmp['voxel'], axis=0), axis=0)	#向voxel_test中添加读取的voxel向量，但是初次读取会出错
        seg_test = np.append(seg_test, np.expand_dims(tmp['seg'], axis=0), axis=0)			#向seg_test中添加读取的seg向量，同样初次读取时会出错
    except ValueError:
        voxel_test = np.expand_dims(tmp['voxel'], axis=0)	#向空矩阵中写入初次读取的voxel	量
        seg_test = np.expand_dims(tmp['seg'], axis=0)	#向空矩阵中写入初次读取的seg量

seg_test = seg_test.astype(int)         #将seg布尔array转换为1/0整数
X_test= voxel_test*seg_test          #将结节抠出来


X_test=X_test.astype(np.float32)
#X_test=X_test/128.-1.
#X_test = np.concatenate((X_test,np.transpose(X_test,(0,2,1,3))),axis=0)    #将训练集的xy转置得到新数据集以扩充数据
#print(X_test.shape)
training_test_size = X_test.shape[0]  #训练数据集的数量
X_test_new=crop(X_test[0],(50,50,50),(32,32,32))

X_test_new=np.expand_dims(X_test_new,axis=0)
print(X_test_new.shape) 
test_batch_size = X_test.shape[0]
for i in tqdm(range(test_batch_size-1),desc='croping'):
    X_test_new=np.append(X_test_new,np.expand_dims(crop(X_test[i+1],(50,50,50),(32,32,32)),axis=0),axis=0)
print(X_test_new.shape)   
del X_test
X_test_new = X_test_new.reshape(X_test_new.shape[0], 32, 32, 32, 1)     #将训练数据集整合成5d张量
print(X_test_new.shape)

croping: 100%|██████████| 116/116 [00:00<00:00, 3036.03it/s]

(1, 32, 32, 32)
(117, 32, 32, 32)
(117, 32, 32, 32, 1)


In [4]:


model = densenet.get_compiled(optimizer=Adam(lr=1.e-3),
                              loss='categorical_crossentropy',
                              metrics=["categorical_accuracy"])
#from keras.models import load_model
#model = load_model('wy.h5')

filepath="best_weight.h5"    
save_folder='test'
#reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.2,patience=2.9, min_lr=0.01)
#callbacks=[reduce_lr]
#early=EarlyStoppingByLossVal(monitor1='val_loss',monitor2='val_acc',value=2.6,verbose=1)
checkpointer = ModelCheckpoint(filepath='tmp/%s/weights.{epoch:02d}.h5' % save_folder, verbose=1,
                               period=1, save_weights_only=False)
best_keeper = ModelCheckpoint(filepath='tmp/%s/best_acc.h5' % save_folder, verbose=1, save_weights_only=False,
                              monitor='val_categorical_accuracy', save_best_only=True, period=1, mode='max')
best_keeper2 = ModelCheckpoint(filepath='tmp/%s/best_loss.h5' % save_folder, verbose=1, save_weights_only=False,
                              monitor='val_loss', save_best_only=True, period=1, mode='min')
early_stopping = EarlyStopping(monitor='val_categorical_accuracy', min_delta=0, mode='max',patience=20, verbose=1)
#checkpoint = ModelCheckpoint(filepath,monitor='val_loss',verbose=0,save_best_only=True,mode='min',period=1)
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.442, patience=5,
                               verbose=1, mode='min', epsilon=1.e-5, cooldown=2, min_lr=0)
#lrs=LearningRateScheduler(schedule=scheduler)
#callbacks_list=[early_stopping, lr_reducer, checkpointer,best_keeper,best_keeper2]
callbacks_list=[lr_reducer, checkpointer,best_keeper,best_keeper2]
training_generator = MixupGenerator(x_train, y_train, batch_size=32, alpha=0.1)()
model.fit_generator(generator=training_generator,
                         steps_per_epoch=365//32+1,shuffle=True,
                         epochs=70,
                         callbacks=callbacks_list,
                         validation_data=(x_val,y_val))

Model hyper-parameters: {'activation': <function <lambda> at 0x7ff3fc8a20e0>, 'bn_scale': True, 'weight_decay': 0.0, 'kernel_initializer': 'he_uniform', 'first_scale': <function <lambda> at 0x7ff3fc8a25f0>, 'dhw': [32, 32, 32], 'k': 16, 'bottleneck': 4, 'compression': 2, 'first_layer': 32, 'down_structure': [4, 4, 4], 'output_size': 2}




__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 32, 1 0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 32, 32, 32, 1 0           input_1[0][0]                    
__________________________________________________________________________________________________
conv3d_1 (Conv3D)               (None, 32, 32, 32, 3 896        








Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




/root/anaconda3/lib/python3.7/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '


Epoch 1/70
12/12 [==============================] - 14s 1s/step - loss: 0.7736 - categorical_crossentropy: 0.7736 - categorical_accuracy: 0.6250 - val_loss: 0.9320 - val_categorical_crossentropy: 0.9320 - val_categorical_accuracy: 0.5300

Epoch 00001: saving model to tmp/test/weights.01.h5

Epoch 00001: val_categorical_accuracy improved from -inf to 0.53000, saving model to tmp/test/best_acc.h5

Epoch 00001: val_loss improved from inf to 0.93203, saving model to tmp/test/best_loss.h5
Epoch 2/70
12/12 [==============================] - 7s 619ms/step - loss: 0.7859 - categorical_crossentropy: 0.7859 - categorical_accuracy: 0.6120 - val_loss: 0.9252 - val_categorical_crossentropy: 0.9252 - val_categorical_accuracy: 0.5300

Epoch 00002: saving model to tmp/test/weights.02.h5

Epoch 00002: val_categorical_accuracy did not improve from 0.53000

Epoch 00002: val_loss improved from 0.93203 to 0.92516, saving model to tmp/test/best_loss.h5
Epoch 3/70
12/12 [==============================] - 7s 

12/12 [==============================] - 7s 618ms/step - loss: 0.7581 - categorical_crossentropy: 0.7581 - categorical_accuracy: 0.6354 - val_loss: 0.9267 - val_categorical_crossentropy: 0.9267 - val_categorical_accuracy: 0.5000

Epoch 00020: saving model to tmp/test/weights.20.h5

Epoch 00020: val_categorical_accuracy did not improve from 0.53000

Epoch 00020: val_loss did not improve from 0.92425
Epoch 21/70
12/12 [==============================] - 7s 618ms/step - loss: 0.7298 - categorical_crossentropy: 0.7298 - categorical_accuracy: 0.6172 - val_loss: 0.9241 - val_categorical_crossentropy: 0.9241 - val_categorical_accuracy: 0.5000

Epoch 00021: saving model to tmp/test/weights.21.h5

Epoch 00021: val_categorical_accuracy did not improve from 0.53000

Epoch 00021: val_loss improved from 0.92425 to 0.92410, saving model to tmp/test/best_loss.h5
Epoch 22/70
12/12 [==============================] - 7s 618ms/step - loss: 0.7145 - categorical_crossentropy: 0.7145 - categorical_accuracy: 

12/12 [==============================] - 7s 618ms/step - loss: 0.7953 - categorical_crossentropy: 0.7953 - categorical_accuracy: 0.6224 - val_loss: 0.9262 - val_categorical_crossentropy: 0.9262 - val_categorical_accuracy: 0.5100

Epoch 00039: saving model to tmp/test/weights.39.h5

Epoch 00039: val_categorical_accuracy did not improve from 0.53000

Epoch 00039: val_loss did not improve from 0.92152
Epoch 40/70
12/12 [==============================] - 7s 618ms/step - loss: 0.7667 - categorical_crossentropy: 0.7667 - categorical_accuracy: 0.6276 - val_loss: 0.9273 - val_categorical_crossentropy: 0.9273 - val_categorical_accuracy: 0.5100

Epoch 00040: ReduceLROnPlateau reducing learning rate to 4.8010260673692075e-09.

Epoch 00040: saving model to tmp/test/weights.40.h5

Epoch 00040: val_categorical_accuracy did not improve from 0.53000

Epoch 00040: val_loss did not improve from 0.92152
Epoch 41/70
12/12 [==============================] - 7s 618ms/step - loss: 0.7138 - categorical_crosse

12/12 [==============================] - 7s 618ms/step - loss: 0.7111 - categorical_crossentropy: 0.7111 - categorical_accuracy: 0.6797 - val_loss: 0.9261 - val_categorical_crossentropy: 0.9261 - val_categorical_accuracy: 0.5100

Epoch 00058: ReduceLROnPlateau reducing learning rate to 4.1457289201574097e-10.

Epoch 00058: saving model to tmp/test/weights.58.h5

Epoch 00058: val_categorical_accuracy did not improve from 0.53000

Epoch 00058: val_loss did not improve from 0.92152
Epoch 59/70
12/12 [==============================] - 7s 618ms/step - loss: 0.7703 - categorical_crossentropy: 0.7703 - categorical_accuracy: 0.6328 - val_loss: 0.9254 - val_categorical_crossentropy: 0.9254 - val_categorical_accuracy: 0.5100

Epoch 00059: saving model to tmp/test/weights.59.h5

Epoch 00059: val_categorical_accuracy did not improve from 0.53000

Epoch 00059: val_loss did not improve from 0.92152
Epoch 60/70
12/12 [==============================] - 7s 618ms/step - loss: 0.7403 - categorical_crosse

KeyboardInterrupt: 

In [5]:
import numpy  
y_pred=model.predict(X_test_new)

numpy.savetxt('new.csv',y_pred, delimiter = ',') 
model.save('first1.h5')

In [ ]:

from keras.models import load_model

model = load_model('tmp/test/weights.40.h5')
import numpy  
y_pred=model.predict(X_test_new)

numpy.savetxt('new1.csv',y_pred, delimiter = ',') 
